<a href="https://colab.research.google.com/github/KanthiKiranS/KK-Repo/blob/master/Pheme_Veracity_1_28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import fastai

In [2]:
from fastai.text import * 

In [3]:
def read_veracity(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 2.0:
            return 'Unverified'
        elif col['label'] == 1.0:
            return 'True'
        elif col['label'] == 0.0:
            return 'False'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [4]:
# For veracity
train = read_veracity('modifiedTest_New.csv', True)
test = read_veracity('modifiedTest_New.csv')

In [5]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [6]:
pd.concat([train['text'], test['text']]).reset_index(drop=True).to_csv('text.csv')

In [7]:
data_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_clas = TextClasDataBunch.from_csv('', 'train.csv', vocab=data_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [8]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [9]:
data_lm = load_data('', 'data_lm_export.pkl')
data_clas = load_data('', 'data_clas_export.pkl', bs=16)

In [10]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.707062,4.176071,0.288264,00:26


In [11]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.956197,3.782415,0.341473,00:30


In [12]:
learn.predict("This is a review about", n_words=10)

'This is a review about the information @honxqp regarding behavior of people in Australia'

In [13]:
learn.save_encoder('ft_enc')

In [14]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (19196 items)
x: TextList
xxbos “ @cnnbrk : xxup cpr being performed on at least 2 after xxmaj sydney siege , which has now ended , xxmaj australia 's xxup abc reports http : / / t.co / xxunk ”,xxbos @richardlionhea @paulgeist45 @bruhnrose @ozzsue @trobinsonnewera too right ; bloody ninjas .,xxbos @physicshenry well it is xxmaj sydney - the selfie absorbed capital of xxmaj australia .,xxbos @julesholman @abc730 police and emergency response units working well with responsible xxunk thankfully this cooperation exists .,xxbos @readingliberty @negijew @nfergus @smh " xxmaj muslim world " shows how fucked your worldview is , designating territory due to religion . xxmaj how liberal
y: CategoryList
True,False,False,True,False
Path: .;

Valid: LabelList (4800 items)
x: TextList
xxbos @pmharper # martinplace # sydneysiege you called xxmaj xxunk was our greatest threat . xxmaj xxunk . xxmaj this is a political movement , not religion .,xxbos

In [15]:
data_clas.show_batch()

text,target
xxbos @troyesivan 😂 🐱 💗 😱 💗 🙏 🐞 👉 👉 💗 👉 👉 🐞 😱 🙏 💗 🙏 😐 😱 follow me pl xxrep 7 z 😍 😍 😭 💗 🐘 xxrep 6 😍 🐘 💗 🐘 🐞 😍 😍 🐘 💗 🐘 📃 💗 🐘 😭 🐽 🙏 🙏 😐 😉 😜 😜 😄 😙 😒 😁 😙 🐳 🐣 🐦 🐟 🐌 🐌 🐛 🐌 🐌 🐌,False
"xxbos .@rconflictnews xxmaj you forgot xxup isil , xxmaj hamas , xxmaj hezbollah , xxmaj islamic xxmaj jihad , xxmaj al xxmaj aqsa xxmaj martyrs xxmaj brigade , xxmaj boko xxmaj haram , xxmaj al xxmaj nusra xxmaj front , xxmaj al - xxmaj shabaab # tcot # tbrs",False
"xxbos "" xxmaj keep xxmaj calm xxmaj and xxmaj do n’t xxmaj speculate : xxmaj how xxmaj to xxmaj be xxmaj helpful xxmaj on xxmaj social xxmaj media xxmaj today "" http : / / t.co / es8yfngjzr # sydneysiege http : / / t.co / kqmpw1jery",False
xxbos @cnnbrk i love xxup cnn \n \n xxmaj katy & & xxmaj bieber embrace xxmaj islam : http : / / t.co / 5azf6v6kxt \n \n xxmaj the xxmaj bible & & xxup nasa embrace xxmaj islam : http : / / t.co / kl7pm8fqct.,True
xxbos xxunk xxup hug xxup you . xxup hug xxup you . xxup hug xxup you xxup rednecks . xxup hug xxup you ' xxup til xxup you xxup cry . xxup hug xxup you xxup motherfuckers . xxup hug xxup you . @mariamveiszadeh,False


In [16]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.610740,0.578727,0.695000,00:22


In [17]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.532819,0.511307,0.746042,00:24


In [18]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.511948,0.485697,0.757708,00:38
1,0.488773,0.438167,0.782917,00:39
2,0.450767,0.397478,0.811458,00:40
3,0.396590,0.358546,0.833333,00:42
4,0.326949,0.329029,0.847083,00:40
5,0.283159,0.318996,0.858958,00:39
6,0.232069,0.315049,0.865833,00:43
7,0.220529,0.323627,0.861042,00:43
8,0.187771,0.321885,0.864792,00:41
9,0.191782,0.323117,0.866458,00:40


In [19]:
l = []
for text in test['text']:
    l.append(str(learn.predict(text)[0]))

In [20]:
from sklearn.metrics import accuracy_score, f1_score

In [21]:
accuracy_score(test['label'], l)

0.9577012835472579

In [22]:
f1_score(test['label'], l, average='macro')

0.9542948270337082

In [23]:
learn.export('veracity.pkl')